In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
import json

In [4]:
theme_nm = [] # 테마명 저장을 위한 리스트 생성
link = [] # 테마별 묶음 페이지 링크를 위한 리스크 생성
k = True # 테마명, 테마별 링크 추출 루프 진행
i = 1 # 페이지 변경
fin_cd = [] # 코드와 테마의 묶음 단위 추출을 위한 리스트 생성
# 테마명 및 테마별 리스트 추출
while k :
    url = 'https://finance.naver.com/sise/theme.naver?&page={}'.format(i)

    headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
        'referer': 'https://finance.naver.com/sise/theme.naver?&page=7','upgrade-insecure-requests': '1'
    }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    path1 = '#contentarea_left > table.type_1.theme > tr > td.col_type1 > a'
    link_path = soup.select(path1)
    
    # 마지막 페이지일 시 해당 루프가 마지막으로 돌도록 k값 변환
    if len(link_path) != 40 :
        k = False
    
    # 추출한 테마명 및 링크값 저장
    for j in range(len(link_path)) : 
        theme_nm.append(link_path[j].text)
        link.append(link_path[j].get('href'))
    i += 1
# 테마별 페이지 내 종목별 코드 추출
for o in range(len(link)) :
    # 링크 이동
    url = 'https://finance.naver.com{}'.format(link[o])

    headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
        'referer': 'https://finance.naver.com/sise/theme.naver?&page=7','upgrade-insecure-requests': '1'
    }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    path2 = '#contentarea > div > table > tbody > tr > td.name > div > a'

    link_path2 = soup.select(path2)
    # 테마별로 리스트 생성을 위해 fin_cd 리스트 초기화

    for k in range(len(link_path2)) :
        # 링크의 마지막 코드만 추출 및 fin_cd 리스트 내 추가
        cd = link_path2[k].get('href')[-6:]
        # 같은 종목이 다른 테마에도 있을 수 있어 코드와 테마를 묶어 unique하게 str로 저장
        list_cd = cd+'_'+theme_nm[o]
        fin_cd.append(list_cd)


# unique한 fin_cd을 구분하여 DataFrame에 저장
fin_theme_cd = pd.DataFrame(columns=['cd','theme'])
cd_list = [] # 종목코드 저장 리스트 생성
theme_list = [] # 테마 저장 리스트 생성
for cd_theme in fin_cd :
    # 위에서 저장한 값을 split 하여 각각 구분
    cd_list.append(cd_theme.split('_')[0])
    theme_list.append(cd_theme.split('_')[1])
# 구분한 종목코드 및 테마명 DataFrame에 추가
fin_theme_cd['cd'] = cd_list
fin_theme_cd['theme'] = theme_list

today = pd.datetime.now()
today_str = str(today)[:10].replace('-','')

path = 'cd_theme_data_{}.csv'.format(today_str)
fin_theme_cd.to_csv(path, encoding="utf-8-sig")

<ipython-input-4-df8d00eebc9e>:66: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime.now()


In [55]:
theme_list = ['자동차 대표주', '항공기부품', '생명보험', '조선','LNG(액화천연가스)', '원자력발전','정유', 'LPG(액화석유가스)',
              '방위산업/전쟁 및 테러','제습기','핵융합에너지','SNS(소셜네트워크서비스)','클라우드 컴퓨팅', '건설 대표주', 
              '겨울','공기청정기', '의료기기','풍력에너지', '태양광에너지', '전력설비', '재난/안전(지진 등)', '도시가스',
              '콜드체인(저온 유통)', '타이어','여행','희귀금속(희토류 등)','여름','페인트','시멘트/레미콘','황사/미세먼지', 
              '태풍 및 장마', '블록체인','인터넷 대표주',  '핀테크(FinTech)', '4차산업 수혜주' ,'T 대표주']
# 테마가 너무 다양해 일부만 진행

In [57]:
# 테마별로 종목의 주식정보 가져오기
# theme_list = ['자동차 대표주', '항공기부품', '생명보험']
fin_theme_cd = pd.read_csv('cd_theme_data_20211021.csv',index_col=0)
fin_theme_cd.cd = fin_theme_cd.cd.apply(lambda x : str(x).zfill(6))
import FinanceDataReader as fdr

for theme_name in theme_list :
    theme_nm = theme_name

    loop_len =  int(fin_theme_cd.cd[fin_theme_cd['theme'] == theme_nm].count())
    df_theme_code_result = pd.DataFrame()

    for i in fin_theme_cd[fin_theme_cd['theme'] == theme_nm].cd :
        search_cd = i
        df_fin = fdr.DataReader(search_cd, '2000','2021-10-20').iloc[:,-3:]
        df_fin['cd'] = search_cd
        df_fin = df_fin.reset_index()

        df_theme_code_result = pd.concat([df_theme_code_result,df_fin])
    # 테마명 내 / 문자가 있으면 path 오류 발생하여 문자 변경
    theme_nm = theme_nm.replace('/','_')
    path = './테마별 주식데이터/{}_list_{}.csv'.format(theme_nm, today_str)
    df_theme_code_result.to_csv(path, encoding="utf-8-sig")